<a href="https://colab.research.google.com/github/HadiAlizadeh1/mci-chatbot/blob/main/task_005.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In the name of Allah


## Befor start Get a Telegram Bot Token

1. **Start a chat with BotFather**  
   - Open Telegram and search for **@BotFather** or visit [BotFather](https://t.me/BotFather).

2. **Create a New Bot**  
   - Type the command: `/newbot`  
   - Choose a **name** for your bot (e.g., `My Awesome Bot`).  
   - Choose a **username** (must end with `bot`, e.g., `my_awesome_bot`).

3. **Receive Your Bot Token**  
   - After creating your bot, you will receive a token like this:
     ```
     1234567890:ABCDEFGHIJKLMNOPQRSTUVWXYZ-abcdefghijklmnopqrstuvwxyz
     ```

4. **Use the Token**  
   - To interact with the Telegram API, use the token in URLs like:
     ```
     https://api.telegram.org/bot<Your_Bot_Token>/METHOD_NAME
     ```

5. **Optional: Configure Your Bot**  
   - Additional configuration commands via BotFather:
     - `/setdescription` – Set bot description
     - `/setcommands` – Set bot commands
     - `/setuserpic` – Set bot avatar
     - `/setprivacy` – Set privacy settings


In [1]:
!pip install vosk # installing vosk model
!wget https://alphacephei.com/vosk/models/vosk-model-fa-0.5.zip # download Persian dataset by Url
!unzip vosk-model-fa-0.5.zip # unzip the dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.2 MB 36.3 MB/s 
     |████████████████████████████████| 112 kB 76.6 MB/s 
  Created wheel for srt: filename=srt-3.5.2-py3-none-any.whl size=22487 sha256=280fc92d46e073b00bf44f3f73fb80f7f41f529d7cff3534d543ab01170be2a9
  Stored in directory: /root/.cache/pip/wheels/54/c4/ec/4604122e072aebb16803c8297b7cd3f4c72073a3ee58738015
Successfully built srt
--2022-10-11 21:00:59--  https://alphacephei.com/vosk/models/vosk-model-fa-0.5.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1130835264 (1.1G) [application/zip]
Saving to: ‘vosk-model-fa-0.5.zip’

vosk-model-fa-0.5.z 100%[===================>]   1.05G  30.5MB/s    in 36s     

2022-10-11 21:01:36 (29.8 MB/s) - ‘vosk-model-fa-0.5.zip’ 

## Connecting to Telegram chat bot

In [1]:
TOKEN = ""

In [2]:
!pip install python-telegram-bot~=13.11

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
! pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Using Vosk model
we use vosk model to change voice to text

In [8]:
from telegram import Update
from telegram.ext import Updater, CallbackContext, MessageHandler, Filters
import librosa as lib
import os,sys # about changing audio format
from pydub import AudioSegment # about changing audio format
import soundfile
import wave # about vosk model function
import sys  # about vosk model function
from vosk import Model, KaldiRecognizer, SetLogLevel # about vosk model function
from pydub import AudioSegment # about vosk model function

def get_voice(update: Update, context: CallbackContext) -> None:
    new_file = context.bot.get_file(update.message.voice.file_id)
    recived_voice=new_file.download("voice_note.ogg")
    update.message.reply_text('صوت شما دریافت شد...لطفا کمی صبر کنید')
    song = AudioSegment.from_ogg("/content/voice_note.ogg") # converting ogg file
    speech_voice = song.export("speech_voice.wav", format="wav") # saving .wav format of voice in speech_voice.wav
    signal,sr=lib.load("/content/speech_voice.wav",sr=48000) # changing sampling_rate to 48000
    soundfile.write('/content/speech_voice.wav',signal,sr)

    sound = AudioSegment.from_wav("/content/speech_voice.wav")
    sound = sound.set_channels(1)
    sound.export("/content/Recorded_voice.wav", format="wav") # crate 1 channel Audio
   

    # vosk vosk
    wf = wave.open("/content/Recorded_voice.wav", "rb")

    model = Model("/content/vosk-model-fa-0.5")

    # You can also init model by name or with a folder path
    # model = Model(model_name="vosk-model-en-us-0.21")
    # model = Model("models/en")

    rec = KaldiRecognizer(model, wf.getframerate())
    rec.SetWords(True)
    rec.SetPartialWords(True)

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            rec.Result()
    update.message.reply_text("متن تشخیص داده شده :")
    update.message.reply_text(rec.FinalResult().split("]")[1].split(":")[1].split("}")[0])

updater = Updater(TOKEN)

updater.dispatcher.add_handler(MessageHandler(Filters.voice , get_voice))

updater.start_polling()
updater.idle()